# Data analysis

Experiment information:
- One million function evaluations
- **sade_remc**: is the best method from HM, but with more evals
- **sade_mc_final**: is sade + MC + ffi9 + rmsd crowding + spicker + hooke jeeves on cluster centroids
- **sade_remc_final**: is the same as above, but REMC instead of MC
- **sade_mc_ffi9_02**: is HM method + forced fragment insertion of size 2 with 0.02 chance of happening per individal per generation
- **sade_remc_ffi9_02**: same as above but with REMC instead of MC

In [13]:
import datetime
import string
import random
import pickle
import time
import sys
import os
import re

import data_utils

import scipy
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
sns.set(style="whitegrid");

In [2]:
root_path = '/home/h3nnn4n/progs/de_supimpa/tools/notebooks/analysis'
base_path = '/home/h3nnn4n/progs/de_supimpa/src'


def reset_path():
    os.chdir(base_path)
    
def reset_to_root_path():
    os.chdir(root_path)

    
reset_to_root_path()
reset_path()

In [3]:
runs = [
    'de_experiment_final',
    'de_sade_remc',
    'de_rosetta',
    'de_ffi',
    'de_experiment_final_8_prot',
    'de_final_1rop_1wqc_1lwy',
    'de_rosetta_all_prots',
    'de_other_experiments_all_prots',
]

In [4]:
protein_blacklist = ['1ab1', '1dfn', '2P5K', '2pmr', '3V1A']

dataset = data_utils.load_all_data(runs)
alldata = data_utils.merge_data(dataset, protein_blacklist=protein_blacklist)

INFO: Loaded 8 experiment runs dataset
removed 5 proteins. Blacklist had 5
[WARN] removing sade_de_mc from 1rop
[WARN] removing sade_mc from 1rop
[WARN] removing sade_de_remc from 1rop
[WARN] removing sade_de_mc from 1l2y
[WARN] removing sade_mc from 1l2y
[WARN] removing sade_de_remc from 1l2y
[WARN] removing sade_de_mc from 2mr9
[WARN] removing sade_mc from 2mr9
[WARN] removing sade_de_remc from 2mr9
[WARN] removing sade_de_mc from 1acw
[WARN] removing sade_mc from 1acw
[WARN] removing sade_de_remc from 1acw
[WARN] removing sade_de_mc from 1utg
[WARN] removing sade_mc from 1utg
[WARN] removing sade_de_remc from 1utg
[WARN] removing sade_de_mc from 1wqc
[WARN] removing sade_mc from 1wqc
[WARN] removing sade_de_remc from 1wqc


# Shapiro-Wilk

In [7]:
def summary_to_dataframe(summary, columns=['raw']):
    proteins = sorted(list(summary.keys()))
    methods = sorted(list(summary[proteins[0]].keys()))
    base_columns = ['protein', 'experiment']
    
    d = {}
    for column in columns + base_columns:
        d[column] = []
        
    for protein in proteins:
        for method in methods:
            for raw in summary[protein][method]['data']['raw']:
                d['raw'].append(raw)

                d['protein'].append(protein)
                d['experiment'].append(method)
                                
    return pd.DataFrame(data=d)

In [58]:
mode = 'best_by_rmsd'
metric = 'scorefxn'
protein = '1zdd'
method = 'sade_remc_final'

summary = data_utils.experiment_summary(alldata, mode=mode, metric=metric, with_raw=True)

# summary[protein][method]['data']['raw']
# summary

In [81]:
def shapiro_table(alpha=0.05, mode='best_by_rmsd', metric='scorefxn'):
    summary = data_utils.experiment_summary(alldata, mode=mode, metric=metric, with_raw=True)
    proteins = sorted(summary.keys())
    methods = sorted(summary[proteins[0]].keys())
    
    print('\\begin{table}')
    print('\\centering')
    column_header = '\\begin{tabular}{r%s} ' % ('|c' * len(methods))
    print(column_header)
    
    methods_header = ' & '.join([ '\\rotatebox[origin=c]{270}{%s}' % m for m in methods]).replace('_', '-')
    
    print('     & %s \\\\ \\hline \\hline' % (methods_header))
    
    for protein in proteins:         
        print('%s & ' % protein, end='')
        
        for index, method in enumerate(methods):
            data = summary[protein][method]['data']['raw']
            
            stats, p = scipy.stats.shapiro(data)
            
            if p < alpha:
                print('$\\bm{%4.4f}$' % p, end='')
            else:
                print('    $%4.4f$ ' % p, end='')
            
            if index < len(methods) -1:
                print(' & ', end='')
            else:
                print(' \\\\ \\hline')   
            
    print('\\end{tabular}')
    metric_dict = { 'scorefxn': 'scorefxn', 'rmsd_after': 'RMSD' }
    metric_string = metric_dict[metric]
    print('\\caption{Shapiro wilk for \\texttt{%s} using %s}' % (mode.replace('_', '-'), metric_string))
    print('\\label{tab:shapiro-wilk-%s-%s}' % (mode.replace('_', '-'), metric_string))
    print('\\end{table}')

In [57]:
modes = ['best_by_rmsd', 'best_by_energy']
metrics = ['scorefxn', 'rmsd_after']

for mode in modes:
    for metric in metrics:
        shapiro_table(alpha=0.05, mode=mode, metric=metric)
        print()

\begin{table}
\centering
\begin{tabular}{r|c|c|c|c|c|c} 
     & \rotatebox[origin=c]{270}{classic-abinitio} & \rotatebox[origin=c]{270}{sade-mc-ffi9-02} & \rotatebox[origin=c]{270}{sade-mc-final} & \rotatebox[origin=c]{270}{sade-remc} & \rotatebox[origin=c]{270}{sade-remc-ffi9-02} & \rotatebox[origin=c]{270}{sade-remc-final} \\ \hline \hline
1acw &     $0.9402$  &     $0.2372$  & $\bm{0.0000}$ &     $0.8959$  &     $0.3392$  & $\bm{0.0003}$ \\ \hline
1ail & $\bm{0.0282}$ & $\bm{0.0208}$ & $\bm{0.0000}$ &     $0.4708$  & $\bm{0.0317}$ & $\bm{0.0000}$ \\ \hline
1crn &     $0.1516$  &     $0.9066$  & $\bm{0.0000}$ &     $0.1411$  &     $0.6835$  & $\bm{0.0000}$ \\ \hline
1enh & $\bm{0.0041}$ &     $0.8152$  & $\bm{0.0000}$ & $\bm{0.0062}$ & $\bm{0.0000}$ & $\bm{0.0000}$ \\ \hline
1l2y &     $0.4865$  &     $0.1457$  & $\bm{0.0000}$ &     $0.1332$  &     $0.4315$  &     $0.0957$  \\ \hline
1rop &     $0.4577$  &     $0.0930$  & $\bm{0.0150}$ &     $0.8606$  & $\bm{0.0144}$ & $\bm{0.0001}$ 

\begin{table}
\centering
\begin{tabular}{r|c|c|c|c|c|c} 
     & \rotatebox[origin=c]{270}{classic-abinitio} & \rotatebox[origin=c]{270}{sade-mc-ffi9-02} & \rotatebox[origin=c]{270}{sade-mc-final} & \rotatebox[origin=c]{270}{sade-remc} & \rotatebox[origin=c]{270}{sade-remc-ffi9-02} & \rotatebox[origin=c]{270}{sade-remc-final} \\ \hline \hline
1acw &     $0.9402$  &     $0.2372$  &     $0.9167$  &     $0.8959$  &     $0.3392$  &     $0.7326$  \\ \hline
1ail & $\bm{0.0282}$ & $\bm{0.0208}$ &     $0.1229$  &     $0.4708$  & $\bm{0.0317}$ &     $0.7205$  \\ \hline
1crn &     $0.1516$  &     $0.9066$  &     $0.3543$  &     $0.1411$  &     $0.6835$  &     $0.2628$  \\ \hline
1enh & $\bm{0.0041}$ &     $0.8152$  &     $0.3685$  & $\bm{0.0062}$ & $\bm{0.0000}$ &     $0.5392$  \\ \hline
1l2y &     $0.4865$  &     $0.1457$  &     $0.2540$  &     $0.1332$  &     $0.4315$  &     $0.8991$  \\ \hline
1rop &     $0.4577$  &     $0.0930$  &     $0.8142$  &     $0.8606$  & $\bm{0.0144}$ &     $0.0892$  

In [80]:
def shapiro_summary(alpha=0.05, mode='best_by_rmsd', metric='scorefxn'):
    summary = data_utils.experiment_summary(alldata, mode=mode, metric=metric, with_raw=True)
    proteins = sorted(summary.keys())
    methods = sorted(summary[proteins[0]].keys())
    
    total_accept_h0 = 0
    total_reject_h0 = 0
    
    for protein in proteins:         
        accept_h0 = 0
        reject_h0 = 0
        
        for index, method in enumerate(methods):
            data = summary[protein][method]['data']['raw']
            
            stats, p = scipy.stats.shapiro(data)
            
            if p < alpha:
                reject_h0 += 1
            else:
                accept_h0 += 1
                
        total_accept_h0 += accept_h0
        total_reject_h0 += reject_h0
                
        print('%s %3d %3d' % (protein, accept_h0, reject_h0))
        
    print('     %3d %3d' % (total_accept_h0, total_reject_h0))
        
        
# modes = ['best_by_rmsd', 'best_by_energy']
# metrics = ['scorefxn', 'rmsd_after']
# for mode in modes:
#     for metric in metrics:
#         print('%s %s' % (mode, metric))
#         shapiro_summary(alpha=0.05, mode=mode, metric=metric)
#         print()
        
mode = 'best_by_rmsd'
metric = 'rmsd_after'

print('%s %s' % (mode, metric))
shapiro_summary(alpha=0.05, mode=mode, metric=metric)
print()

mode = 'best_by_energy'
metric = 'scorefxn'

print('%s %s' % (mode, metric))
shapiro_summary(alpha=0.05, mode=mode, metric=metric)
print()

best_by_rmsd rmsd_after
1acw   6   0
1ail   5   1
1crn   3   3
1enh   5   1
1l2y   3   3
1rop   5   1
1utg   4   2
1wqc   2   4
1zdd   1   5
2mr9   3   3
      37  23

best_by_energy scorefxn
1acw   6   0
1ail   3   3
1crn   6   0
1enh   3   3
1l2y   6   0
1rop   5   1
1utg   5   1
1wqc   5   1
1zdd   2   4
2mr9   5   1
      46  14



# Kruskal-Wallis

In [82]:
def kruskal_wallis_table(alpha=0.05, mode='best_by_rmsd', metric='rmsd_after'):
    summary = data_utils.experiment_summary(alldata, mode=mode, metric=metric, with_raw=True)
    proteins = sorted(summary.keys())
    methods = sorted(summary[proteins[0]].keys())
    
    print('\\begin{table}')
    print('\\centering')
    print('\\begin{tabular}{r|c}')

    methods_header = 'Protein & $p$-value'
    
    print('%s \\\\ \\hline \\hline' % (methods_header))
    
    for protein in proteins:         
        print('%s & ' % protein, end='')
        
        data = [
            summary[protein][method]['data']['raw'] for method in methods
        ]
            
        stats, p = scipy.stats.kruskal(*data)
        
        if p < alpha:
            print('$\\bm{%4.4f}$' % p, end='')
        else:
            print('    $%4.4f$ ' % p, end='')
            
        print(' \\\\ \\hline')   
            
    print('\\end{tabular}')
    metric_dict = { 'scorefxn': 'scorefxn', 'rmsd_after': 'RMSD' }
    metric_string = metric_dict[metric]
    print('\\caption{Kruskal-Wallis for \\texttt{%s} using %s}' % (mode.replace('_', '-'), metric_string))
    print('\\label{tab:kruskal-wallis-wilk-%s-%s}' % (mode.replace('_', '-'), metric_string))
    print('\\end{table}')

In [83]:
kruskal_wallis_table(alpha=0.05, mode='best_by_rmsd', metric='rmsd_after')
print()
kruskal_wallis_table(alpha=0.05, mode='best_by_energy', metric='scorefxn')

\begin{table}
\centering
\begin{tabular}{r|c}
Protein & $p$-value \\ \hline \hline
1acw & $\bm{0.0000}$ \\ \hline
1ail & $\bm{0.0000}$ \\ \hline
1crn & $\bm{0.0000}$ \\ \hline
1enh & $\bm{0.0000}$ \\ \hline
1l2y & $\bm{0.0000}$ \\ \hline
1rop & $\bm{0.0000}$ \\ \hline
1utg & $\bm{0.0000}$ \\ \hline
1wqc & $\bm{0.0000}$ \\ \hline
1zdd &     $0.4145$  \\ \hline
2mr9 & $\bm{0.0000}$ \\ \hline
\end{tabular}
\caption{Mann-Whitney for \texttt{best-by-rmsd} using RMSD}
\label{tab:mann-whitney-wilk-best-by-rmsd-RMSD}
\end{table}

\begin{table}
\centering
\begin{tabular}{r|c}
Protein & $p$-value \\ \hline \hline
1acw & $\bm{0.0177}$ \\ \hline
1ail & $\bm{0.0000}$ \\ \hline
1crn & $\bm{0.0000}$ \\ \hline
1enh & $\bm{0.0001}$ \\ \hline
1l2y &     $0.2930$  \\ \hline
1rop & $\bm{0.0000}$ \\ \hline
1utg & $\bm{0.0000}$ \\ \hline
1wqc & $\bm{0.0001}$ \\ \hline
1zdd &     $0.1558$  \\ \hline
2mr9 & $\bm{0.0000}$ \\ \hline
\end{tabular}
\caption{Mann-Whitney for \texttt{best-by-energy} using scorefxn}

# Mann-Whitney

In [92]:
def mann_whitney_list(alpha=0.05, mode='best_by_rmsd', metric='rmsd_after'):
    summary = data_utils.experiment_summary(alldata, mode=mode, metric=metric, with_raw=True)
    proteins = sorted(summary.keys())
    
    methods = sorted(summary[proteins[0]].keys())
    
    for protein in proteins:
        for i in range(len(methods)):
            method_a = methods[i]
            for j in range(i + 1, len(methods)):
                method_b = methods[j]
                p, mean_a, mean_b = mann_whitney_sublist(summary, method_a, method_b, protein)
                
                if p > alpha:
                    continue
                    
                if mean_a < mean_b:
                    print('%5s %20s %20s %f' % (protein, method_a, method_b, p))
                else:
                    print('%5s %20s %20s %f' % (protein, method_b, method_a, p))
        print()
        

def mann_whitney_sublist(data, method_a, method_b, protein):
    data_a = data[protein][method_a]['data']['raw']
    data_b = data[protein][method_b]['data']['raw']
    _, p = scipy.stats.mannwhitneyu(data_a, data_b)
    
    mean_a = data[protein][method_a]['data']['mean']
    mean_b = data[protein][method_b]['data']['mean']
    
    return p, mean_a, mean_b


# mann_whitney_table(alpha=0.05, mode='best_by_rmsd', metric='rmsd_after')

In [123]:
def mann_whitney_master(alpha=0.05, mode='best_by_rmsd', metric='rmsd_after'):
    summary = data_utils.experiment_summary(alldata, mode=mode, metric=metric, with_raw=True)
    proteins = sorted(summary.keys())
    methods = sorted(summary[proteins[0]].keys())
    
    for protein in proteins:
        table_header(methods)
        mann_whitney_table(summary[protein], alpha=alpha)
        table_footer(mode, metric, protein)
        
        print()
        
        break
        
        
def mann_whitney_table(data, alpha=0.5):
    methods = sorted(data.keys())
    
    for i in range(len(methods)):
        method_a = methods[i]
        print('%20s & ' % method_a.replace('_', '-'), end='')
        
        results = []
        
        for j in range(0, len(methods)):
            method_b = methods[j]
            p, mean_a, mean_b = apply_mann_whitney(data, method_a, method_b, protein)

            if i == j:
                results.append(' -    ')
                continue
              
            if p > alpha:  # Draws
                results.append('$%4.4f$     ' % p)
            elif mean_a < mean_b:  # Wins
                results.append('$\\bm{%4.4f}$' % p)
            else:  # Loses
                results.append('$%4.4f$     ' % p)
            
        print('%s' % ' & '.join(results), end='')
        print(' \\\\ \\hline')
    print('\\hline')
                
                
def table_header(methods):    
    print('\\begin{table}')
    print('\\centering')
    column_header = '\\begin{tabular}{r%s} ' % ('|c' * len(methods))
    print(column_header)
    
    methods_header = ' & '.join([ '\\rotatebox[origin=c]{270}{%s}' % m for m in methods]).replace('_', '-')
    
    print('     & %s \\\\ \\hline \\hline' % (methods_header))
    
    
                
def table_footer(mode, metric, protein):
    print('\\end{tabular}')
    metric_dict = { 'scorefxn': 'scorefxn', 'rmsd_after': 'RMSD' }
    metric_string = metric_dict[metric]
    print('\\caption{Mann-Whitney for %s \\texttt{%s} using %s}' % (protein, mode.replace('_', '-'), metric_string))
    print('\\label{tab:mann-whitney-wilk-%s-%s}' % (mode.replace('_', '-'), metric_string))
    print('\\end{table}')

                
def apply_mann_whitney(data, method_a, method_b, protein):
    data_a = data[method_a]['data']['raw']
    data_b = data[method_b]['data']['raw']
    _, p = scipy.stats.mannwhitneyu(data_a, data_b)
    
    mean_a = data[method_a]['data']['mean']
    mean_b = data[method_b]['data']['mean']
    
    return p, mean_a, mean_b


mann_whitney_master(alpha=0.05, mode='best_by_rmsd', metric='rmsd_after')

\begin{table}
\centering
\begin{tabular}{r|c|c|c|c|c|c} 
     & \rotatebox[origin=c]{270}{classic-abinitio} & \rotatebox[origin=c]{270}{sade-mc-ffi9-02} & \rotatebox[origin=c]{270}{sade-mc-final} & \rotatebox[origin=c]{270}{sade-remc} & \rotatebox[origin=c]{270}{sade-remc-ffi9-02} & \rotatebox[origin=c]{270}{sade-remc-final} \\ \hline \hline
    classic-abinitio &  -     & $0.1769$      & $0.0000$      & $0.0000$      & $0.0000$      & $0.0000$      \\ \hline
     sade-mc-ffi9-02 & $0.1769$      &  -     & $0.0000$      & $0.0020$      & $0.0028$      & $0.0000$      \\ \hline
       sade-mc-final & $\bm{0.0000}$ & $\bm{0.0000}$ &  -     & $\bm{0.0000}$ & $\bm{0.0000}$ & $0.3000$      \\ \hline
           sade-remc & $\bm{0.0000}$ & $\bm{0.0020}$ & $0.0000$      &  -     & $0.4684$      & $0.0000$      \\ \hline
   sade-remc-ffi9-02 & $\bm{0.0000}$ & $\bm{0.0028}$ & $0.0000$      & $0.4684$      &  -     & $0.0000$      \\ \hline
     sade-remc-final & $\bm{0.0000}$ & $\bm{0.0000}$ & $